# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
%load_ext dotenv
%dotenv 

The command `%run update_path.py` runs a script from `01_materials/labs` that adds the repository's `./05_src/` directory to the Notebook's kernel path. This way, we can use our modules like DataManager within the notebook.

In [2]:
from pathlib import Path
import os
notebook_dir = Path.cwd()
src_path = (notebook_dir / "../../01_materials/labs").resolve()
update_path = os.path.join(src_path, 'update_path.py')
%run $update_path

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import dask.dataframe as dd
import pandas as pd

from utils.logger import get_logger
_logs = get_logger(__name__)


Create an object `dm` of class `DataManager`


In [4]:
from stock_prices.data_manager import DataManager
dm = DataManager()

Call the method load_prices will read the parquet files and save the contents into field `price_dd`

In [5]:
dm.load_prices()
dd_px = dm.price_dd
dd_px

2026-01-18 18:08:30,430, data_manager.py, 144, INFO, Loading price data from ../../05_src/data/prices/


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
npartitions=90,,,,,,,,,
ACIU,datetime64[ns],float64,float64,float64,float64,float64,int64,string,int32
ACN,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...


Call method `create_features` to create features from price data.
Add `Adj_Close_lag_1` and `hi_lo_range` since they are are not part of the features created in `create_features()`

In [6]:
dm.create_features()
dd_feat = dm.features
dd_feat = dd_feat.assign(
    Adj_Close_lag_1 = lambda x: x['Adj Close'].shift(1)
)

dd_feat = dd_feat.assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
)
dd_feat


2026-01-18 18:08:37,085, data_manager.py, 153, INFO, Creating features


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Returns,Adj_Close_lag_1,hi_lo_range
npartitions=90,,,,,,,,,,,,,
ACIU,object,object,object,object,object,object,object,object,object,object,object,object,object
ACN,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.
+ After all data transformations are done, convert the Dask data frame to a pandas data frame. 

In [7]:
# Convert the Dask DataFrame to a pandas DataFrame
dd_feat['returns_rolling_mean_10'] = dd_feat['Returns'].rolling(window=10).mean()
pd_feat = dd_feat.compute()
pd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Returns,Adj_Close_lag_1,hi_lo_range,returns_rolling_mean_10
ticker,,,,,,,,,,,,,,
ACIU,2016-09-23,14.110000,16.799999,14.110000,15.660000,15.660000,3775900.0,ACIU.csv,2016,NaN,NaN,NaN,2.690000,NaN
ACIU,2016-09-26,15.600000,17.400000,15.500000,16.500000,16.500000,1008300.0,ACIU.csv,2016,15.660000,0.053640,15.660000,1.900000,NaN
ACIU,2016-09-27,16.780001,18.364000,16.389999,18.209999,18.209999,531500.0,ACIU.csv,2016,16.500000,0.103636,16.500000,1.974001,NaN
ACIU,2016-09-28,19.040001,19.969999,18.600000,19.299999,19.299999,474400.0,ACIU.csv,2016,18.209999,0.059857,18.209999,1.369999,NaN
ACIU,2016-09-29,19.600000,19.700001,17.219999,17.540001,17.540001,335400.0,ACIU.csv,2016,19.299999,-0.091192,19.299999,2.480001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.060000,4.530000,3.880000,4.510000,4.510000,1668500.0,ZIXI.csv,2020,3.080000,0.464286,4.000000,0.650000,0.228450
ZIXI,2020-03-27,4.490000,4.710000,4.100000,4.600000,4.600000,1146800.0,ZIXI.csv,2020,3.040000,0.513158,4.510000,0.610000,0.224684
ZIXI,2020-03-30,4.830000,4.870000,4.440000,4.640000,4.640000,1212000.0,ZIXI.csv,2020,3.060000,0.516340,4.600000,0.430000,0.241286


## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.